In [97]:
import pandas as pd
import numpy as np

data = pd.read_csv("D:\CPU.csv")

In [98]:
data.head()

,Date,Time,CPU Utilization
0,01 January 2020,00:00:00,30
1,01 January 2020,00:05:00,27
2,01 January 2020,00:10:00,22
3,01 January 2020,00:15:00,24
4,01 January 2020,00:20:00,21


In [99]:
data["Date"] = data["Date"].astype(str)+" "+data["Time"].astype(str)


In [100]:
data["Date"] = pd.to_datetime(data["Date"], dayfirst= True)

In [101]:
data.head()

,Date,Time,CPU Utilization
0,2020-01-01 00:00:00,00:00:00,30
1,2020-01-01 00:05:00,00:05:00,27
2,2020-01-01 00:10:00,00:10:00,22
3,2020-01-01 00:15:00,00:15:00,24
4,2020-01-01 00:20:00,00:20:00,21


In [102]:
data = data.drop(["Time"], axis = 1)

In [103]:
data.head()

,Date,CPU Utilization
0,2020-01-01 00:00:00,30
1,2020-01-01 00:05:00,27
2,2020-01-01 00:10:00,22
3,2020-01-01 00:15:00,24
4,2020-01-01 00:20:00,21


In [104]:
data = data.rename(columns={"CPU Utilization": "cpu_data"}, errors="raise")

In [105]:
#data.to_csv("D:\\projects\\myvenv\\demand_forecast-dev\\demand_forecast-univarient\\demand_forecast-univarient _cpu\\demand_forecast-univarient\\sales_forecast\\cpu_data.csv")

In [106]:
data["hour"] = data["Date"].dt.hour
data["min"] = data["Date"].dt.minute
data["day"] = data["Date"].dt.day
data["month"] = data["Date"].dt.month
data["year"] = data["Date"].dt.year

In [107]:
data.head()

,Date,cpu_data,hour,min,day,month,year
0,2020-01-01 00:00:00,30,0,0,1,1,2020
1,2020-01-01 00:05:00,27,0,5,1,1,2020
2,2020-01-01 00:10:00,22,0,10,1,1,2020
3,2020-01-01 00:15:00,24,0,15,1,1,2020
4,2020-01-01 00:20:00,21,0,20,1,1,2020


In [108]:
data = data.drop(["Date"], axis = 1)

In [109]:
length = len(data)
print(length)
trainlength = np.round(length*0.7)
print(trainlength)

9125
6388.0


In [110]:
train = data[data.index <= trainlength]
test = data[data.index > trainlength]

In [111]:
require_cols = ["day","month","min"]
x_train= train[require_cols]
target_col = ["cpu_data"]
y_train= train[target_col]

require_cols = ["day","month","min"]
x_test = test[require_cols]
target_col = ["cpu_data"]
y_test= test[target_col]
complete_x = data[require_cols]

In [112]:
from sklearn.ensemble import RandomForestRegressor 

In [113]:
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0) 

In [114]:
regressor.fit(x_train, y_train)

C:\Users\ketank\AppData\Local\Continuum\anaconda3\envs\conda_env\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [115]:
regressor.feature_importances_

array([0.88221902, 0.        , 0.11778098])

In [116]:
x_train.columns

Index(['day', 'month', 'min'], dtype='object')

In [117]:
pred = regressor.predict(x_test)

In [118]:
y_true, y_pred = np.array(y_test), np.array(pred)
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
print(mape)

32.502961106340486


In [119]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(y_true, y_pred))
print(rms)

10.592623186452377


In [120]:
require_cols = ["day","month","year","hour","min"]
comlete_x= data[require_cols]

In [121]:
complete_pred = regressor.predict(complete_x)
data["predicted_cpu_usage"] = complete_pred

In [122]:
data["error_cpu"] = (data["cpu_data"] - data["predicted_cpu_usage"])

In [123]:
data

,cpu_data,hour,min,day,month,year,predicted_cpu_usage,error_cpu
0,30,0,0,1,1,2020,37.399967,-7.399967
1,27,0,5,1,1,2020,36.724172,-9.724172
2,22,0,10,1,1,2020,35.720997,-13.720997
3,24,0,15,1,1,2020,34.062828,-10.062828
4,21,0,20,1,1,2020,34.508462,-13.508462
...,...,...,...,...,...,...,...,...
9120,46,15,59,2,2,2020,36.283699,9.716301
9121,43,16,4,2,2,2020,34.381994,8.618006
9122,44,16,9,2,2,2020,36.938721,7.061279
9123,46,16,14,2,2,2020,37.240099,8.759901


In [124]:
def find_outliers_tukey(x):
    """
    This function takes Actual-Expected as arg and return outliers indices , values and threshold.
    :param: pandas data frame  object as Difference of Actual - Predicted
    :return value1: indices
    :return value2: outlier values and threshold values upper and lower limit
    :rtype : List. outlier_indices and outlier_values returned as List
     """
    q1 = np.percentile(x, 25)
    q3 = np.percentile(x, 75)
    iqr = q3 - q1
    #floor = q1 - 1.5 * iqr
    ceiling = q3 + 1.0 * iqr
    print("ceiling",ceiling)
    outlier_indices = list(x.index[(x > ceiling)])
    outlier_values = list(x[outlier_indices])
    return outlier_indices, outlier_values, ceiling

In [125]:
def get_anomaly(output_df):
    anomaly_df = output_df.copy()
    #df = anomaly_df["yhat"]
    #for col in df.columns:
    outlier_indices, outlier_values, ceiling = find_outliers_tukey(anomaly_df["error_cpu"])
    temp_col = "anomaly"
    print("index",outlier_indices)
    print("out",outlier_values)
    data[temp_col] = np.where(anomaly_df["error_cpu"].isin(outlier_values),1,0)
    return data

In [126]:
data = get_anomaly(data)

ceiling 26.240300673725464
index [3614, 3615, 3617, 3618, 3619]
out [26.913190189983673, 29.000951255640913, 27.798348537215468, 29.239537653823966, 27.32613895524765]


In [127]:
data.head()

,cpu_data,hour,min,day,month,year,predicted_cpu_usage,error_cpu,anomaly
0,30,0,0,1,1,2020,37.399967,-7.399967,0
1,27,0,5,1,1,2020,36.724172,-9.724172,0
2,22,0,10,1,1,2020,35.720997,-13.720997,0
3,24,0,15,1,1,2020,34.062828,-10.062828,0
4,21,0,20,1,1,2020,34.508462,-13.508462,0


In [128]:
anomaly_list = data["anomaly"].tolist()
threat_list = []
#print("Start",anomaly_list)
for i in range(len(anomaly_list)-1):
    if(anomaly_list[i] == anomaly_list[i+1]):
        if anomaly_list[i] == 1:
        #counter+=2 
            #print(anomaly_list[i])
            #print(anomaly_list[i+1])
            threat_list.append(i)
            threat_list.append(i+1)
            
         
print(type(anomaly_list))
print("--------------------------------------------------")
#print(threat_list)
threat_list2 = list(dict.fromkeys(threat_list))
#print(threat_list2)
for i in threat_list2:
    data.loc[i, 'Threat'] = "Threat"

#print(anomaly_df)
anomaly_df = data.replace(np.nan, '-', regex=True)

anomaly_df.to_csv("Final_output.csv")

<class 'list'>
--------------------------------------------------
